# 影像處理 HW2 ( histogram equalization )
## 基本資料
系級 : 資工 115 
學號 : 41147047S
作者 : 黃國展

### 環境建置以及套件引用

In [2]:
## 安裝套件
# %pip install gradio opencv-python numpy

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import os

In [4]:
def compute_gray_histogram(image):
    """
    計算灰階圖片的直方圖。
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # 從彩色圖像 ( RGB ) 轉換為灰階圖像
    hist_gray = cv2.calcHist([gray], [0], None, [256], [0, 256]).flatten() # 計算灰階直方圖
    return gray, hist_gray

def histogram_equalization(image):
    """
    進行直方圖均衡化，透過灰階影像作為參考來調整彩色影像。
    """
    # 轉換為灰階
    gray, hist_gray = compute_gray_histogram(image)
    
    # 對灰階圖像應用直方圖均衡化
    gray_eq = cv2.equalizeHist(gray)
    hist_gray_eq = cv2.calcHist([gray_eq], [0], None, [256], [0, 256]).flatten()
    
    # 避免 G 為零的情況，避免除零錯誤
    gray = np.where(gray == 0, 1, gray)
    
    # 調整彩色影像的 RGB 通道
    b, g, r = cv2.split(image)
    b = np.clip((b.astype(np.float32) * gray_eq / gray), 0, 255).astype(np.uint8)
    g = np.clip((g.astype(np.float32) * gray_eq / gray), 0, 255).astype(np.uint8)
    r = np.clip((r.astype(np.float32) * gray_eq / gray), 0, 255).astype(np.uint8)
    
    # 合併回彩色圖像
    equalized_image = cv2.merge([b, g, r])
    
    return image, equalized_image, hist_gray, hist_gray_eq

def plot_histogram(hist, title, filename):
    """
    繪製灰階直方圖。
    """
    plt.figure()
    plt.title(title)
    plt.xlabel("Gray Level")
    plt.ylabel("Pixel Count")
    plt.bar(range(256), hist, width=1.0, color='black')
    plt.xlim([0, 255])
    
    # 轉換成圖像格式
    plt.tight_layout()
    save_path = os.path.join(os.getcwd(), filename)
    plt.savefig(save_path)
    plt.close()
    return save_path



### 建立 gradio ( GUI 介面以方便操作 )

In [5]:
def gradio_interface(image):
    """
    Gradio 介面函數，處理使用者上傳的圖片並顯示直方圖及均衡化結果。
    """
    original_image, equalized_image, hist_gray, hist_gray_eq = histogram_equalization(image)
    
    # 繪製原始灰階直方圖
    original_hist_path = plot_histogram(hist_gray, "Original Gray Histogram", "original_gray_hist.png")
    
    # 繪製均衡化後的灰階直方圖
    equalized_hist_path = plot_histogram(hist_gray_eq, "Equalized Gray Histogram", "equalized_gray_hist.png")
    
    return original_image, original_hist_path, equalized_image, equalized_hist_path



# 創建 Gradio 介面
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Image(type="numpy"),
    outputs=[gr.Image(type="numpy", label="原始彩色圖像"),
             gr.Image(label="原始直方圖"),
             gr.Image(type="numpy", label="均衡化後圖像"),
             gr.Image(label="均衡化後直方圖")],
    title="直方圖均衡化展示 (彩色圖像)",
    description="上傳圖片以查看其 RGB 直方圖，並進行直方圖均衡化處理。"
)

# 啟動 Gradio 應用
iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
